In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -q -U transformers==4.55.2 trl==0.21.0 datasets==4.0.0 huggingface_hub==0.34.4 accelerate==1.10.0 peft==0.17.0 bitsandbytes==0.47.0 wandb==0.21.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 88.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00

In [4]:
import os
import torch
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, setup_chat_format
from transformers import BitsAndBytesConfig

2025-10-02 11:00:41.946020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759402842.287138      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759402842.386152      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

hf_token = UserSecretsClient().get_secret("HF_TOKEN")

login(token=hf_token)

In [6]:
# MODEL_NAME = "CohereLabs/aya-23-8B"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
# NEW_MODEL_REPO = "Yanmife/aya-wiki-data"
NEW_MODEL_REPO = "Yanmife/nllb-menyo"

WANDB_PROJECT = "Fine-tuning-nllb-menyo"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang="eng_Latn", tgt_lang="yor_Latn", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [9]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    # quantization_config=bnb_config,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=False,
    low_cpu_mem_usage=True,
)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,            # rank
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

In [11]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 618,612,736 || trainable%: 0.5721


In [12]:
print("Pad token:", tokenizer.pad_token, tokenizer.pad_token_id)
print("BOS token:", tokenizer.bos_token, tokenizer.bos_token_id)
print("EOS token:", tokenizer.eos_token, tokenizer.eos_token_id)

Pad token: <pad> 1
BOS token: <s> 0
EOS token: </s> 2


In [14]:
print(model.config.decoder_start_token_id)

2


In [15]:
!git clone https://github.com/uds-lsv/menyo-20k_MT.git

Cloning into 'menyo-20k_MT'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 151 (delta 59), reused 71 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (151/151), 4.43 MiB | 15.08 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [16]:
from datasets import load_dataset
import os

data_path = "menyo-20k_MT/data"

# Load train/dev/test
menyo_ds = load_dataset(
    "csv",
    data_files={
        "train": os.path.join(data_path, "train.tsv"),
        "dev": os.path.join(data_path, "dev.tsv"),
        "test": os.path.join(data_path, "test.tsv"),
    },
    delimiter="\t",  # important: TSV files
)

print(menyo_ds)

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 10070
    })
    dev: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 3397
    })
    test: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 6633
    })
})


In [17]:
from datasets import concatenate_datasets

train_ds = concatenate_datasets([menyo_ds["train"], menyo_ds["test"]])

# Keep dev for evaluation
test_ds = menyo_ds["dev"]

print(f"Train size: {len(train_ds)}")
print(f"Dev size: {len(test_ds)}")

Train size: 16703
Dev size: 3397


In [18]:
train_ds

Dataset({
    features: ['English', 'Yoruba'],
    num_rows: 16703
})

In [19]:
train_ds[1]

{'English': 'This work is licensed under a Creative Commons Attribution 4.0 International License.',
 'Yoruba': 'Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribution 4.0 International License.'}

In [21]:
def formatting_func(example):
    # Source text
    source_text = example["English"]
    target_text = example["Yoruba"]

    # Tokenize source and target without padding
    model_inputs = tokenizer(
        source_text,
        truncation=True,
        max_length=512,
    )

    labels = tokenizer(
        target_text,
        truncation=True,
        max_length=256,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [22]:
tokenized_train = train_ds.map(formatting_func, remove_columns=train_ds.column_names)
tokenized_test = test_ds.map(formatting_func, remove_columns=test_ds.column_names)

Map:   0%|          | 0/16703 [00:00<?, ? examples/s]

Map:   0%|          | 0/3397 [00:00<?, ? examples/s]

In [23]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16703
})

In [24]:
sample = tokenized_train[0]
print("labels:", sample["labels"])

labels: [256047, 13596, 15205, 6455, 213934, 158, 106666, 81464, 248130, 2]


In [25]:
import wandb

wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")

wandb.login(key=wandb_key)

wandb.init(project=WANDB_PROJECT)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egbewaleyanmife (yanmife-work-1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [26]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

In [27]:
training_args = TrainingArguments(
    output_dir="./nllb-menyo",       # where to save checkpoints
    per_device_train_batch_size=8,     # adjust based on GPU memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,     # for effective larger batch
    num_train_epochs=50,                # adjust based on dataset size
    learning_rate=1e-4,
    
    lr_scheduler_type="cosine",  # cosine restart
    # num_cycles=2,                   # number of restarts
    warmup_steps=1000,               # gradual warmup

    logging_steps=200,
    save_steps=200,
    eval_steps=200,
    eval_strategy="steps",       # evaluate every N steps
    save_total_limit=2,                # keep last 2 checkpoints
    fp16=True,                         # use mixed precision if GPU supports
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [28]:
trainer = Trainer(
    model=model,                   # your Aya model
    args=training_args,
    train_dataset=tokenized_train,  # tokenized MENYO-20k train split
    eval_dataset=tokenized_test,     # tokenized dev split
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],

)

/tmp/ipykernel_36/4186336362.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
!rm -rf /content/Yanmife/nllb-menyo
!rm -rf /content/nllb-menyo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [31]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
200,4.766500,4.068234
400,3.428000,2.884873
600,2.502900,2.329353
800,2.287000,2.259380
1000,2.217800,2.221076
1200,2.169800,2.191827
1400,2.143000,2.169611
1600,2.115300,2.155494
1800,2.086000,2.139376
2000,2.073300,2.127661


{"timestamp":"2025-10-02T16:37:03.046853Z","level":"WARN","fields":{"message":"Status Code: 504. Retrying...","request_id":""},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-02T16:37:03.046916Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 1.649233612s before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}
{"timestamp":"2025-10-02T16:37:03.074737Z","level":"WARN","fields":{"message":"Status Code: 504. Retrying...","request_id":""},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-10-02T16:37:03.074760Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 1.283384033s before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


TrainOutput(global_step=12800, training_loss=1.9839582228660584, metrics={'train_runtime': 27325.9367, 'train_samples_per_second': 30.563, 'train_steps_per_second': 0.478, 'total_flos': 9.25455633007657e+16, 'train_loss': 1.9839582228660584, 'epoch': 49.04214559386973})

In [32]:
model.push_to_hub(NEW_MODEL_REPO)

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  nllb-menyo/adapter_model.safetensors  : 100%|##########| 14.2MB / 14.2MB            

CommitInfo(commit_url='https://huggingface.co/Yanmife/nllb-menyo/commit/6526b59a89c09f9d201003ee07ec6f484e8f1ea9', commit_message='Upload model', commit_description='', oid='6526b59a89c09f9d201003ee07ec6f484e8f1ea9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yanmife/nllb-menyo', endpoint='https://huggingface.co', repo_type='model', repo_id='Yanmife/nllb-menyo'), pr_revision=None, pr_num=None)

In [33]:
tokenizer.push_to_hub(NEW_MODEL_REPO)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  nllb-menyo/sentencepiece.bpe.model    : 100%|##########| 4.85MB / 4.85MB            

  nllb-menyo/tokenizer.json             : 100%|##########| 32.2MB / 32.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Yanmife/nllb-menyo/commit/6526b59a89c09f9d201003ee07ec6f484e8f1ea9', commit_message='Upload tokenizer', commit_description='', oid='6526b59a89c09f9d201003ee07ec6f484e8f1ea9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yanmife/nllb-menyo', endpoint='https://huggingface.co', repo_type='model', repo_id='Yanmife/nllb-menyo'), pr_revision=None, pr_num=None)

In [34]:
trainer.push_to_hub(NEW_MODEL_REPO)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...orking/nllb-menyo/training_args.bin: 100%|##########| 5.30kB / 5.30kB            

  ...events.1759402945.5fbb1a2d6e80.36.0: 100%|##########| 36.4kB / 36.4kB            

  .../nllb-menyo/sentencepiece.bpe.model: 100%|##########| 4.85MB / 4.85MB            

  ...llb-menyo/adapter_model.safetensors: 100%|##########| 14.2MB / 14.2MB            

  ...e/working/nllb-menyo/tokenizer.json: 100%|##########| 32.2MB / 32.2MB            

CommitInfo(commit_url='https://huggingface.co/Yanmife/nllb-menyo/commit/cc9091dc7b7858955eedf6afbbf52dae2a21b2de', commit_message='Yanmife/nllb-menyo', commit_description='', oid='cc9091dc7b7858955eedf6afbbf52dae2a21b2de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Yanmife/nllb-menyo', endpoint='https://huggingface.co', repo_type='model', repo_id='Yanmife/nllb-menyo'), pr_revision=None, pr_num=None)

In [35]:
from peft import PeftModel, PeftConfig

In [36]:
yor_model = PeftModel.from_pretrained(base_model, NEW_MODEL_REPO)

adapter_config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [37]:
config = PeftConfig.from_pretrained(NEW_MODEL_REPO)

In [44]:
def encode_input(text, tokenizer):
    return tokenizer(text, return_tensors="pt")

In [45]:
source_lang = "eng_Latn"
target_lang = "yor_Latn"

tokenizer.src_lang = source_lang
target_lang_id = tokenizer.convert_tokens_to_ids(target_lang)  # BOS token

In [46]:
def translate(sentence, max_length=128):
    # Encode input
    encoded = tokenizer(sentence, return_tensors="pt").to("cuda")
    
    # Generate translation
    generated = yor_model.generate(
        **encoded,
        forced_bos_token_id=target_lang_id,
        max_length=max_length,
        num_beams=5,
        early_stopping=True,
    )
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)


In [49]:
sentence = "Would you marry me?"
translation = translate(sentence)
print("Input:", sentence)
print("Translation:", translation)

Input: Would you marry me?
Translation: Ṣé wàá fẹ́ mi?
